# <center> Forecasting City Finances - Modeling</center>


## Import Packages
The needed packages for treating the data are imported below:

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

## Load the data
We now load the data that we cleaned in the data wrangling step.

In [2]:
pwd_data = '/Users/Varishth/Desktop/Springboard_Projects/capstone_two/data/timeseries_data.csv'
df_TS = pd.read_csv(pwd_data)
df_TS.head()

,year,state_city,rev_total
0,1977,AK: Anchorage,5342.24
1,1978,AK: Anchorage,5948.99
2,1979,AK: Anchorage,6158.68
3,1980,AK: Anchorage,5654.93
4,1981,AK: Anchorage,6192.83


## Slight field adjusments
We have two slight adjusments to make: 1) make the year the index of the dataframe, and 2) pivot the dataframe to ensure that the index of the dataframe is only the unique datetime values.

In [3]:
# make the year column as the index
df_TS.index = pd.to_datetime(df_TS.year, format='%Y')
df_TS.drop("year",axis=1, inplace=True)

df_TS.head()

,state_city,rev_total
year,,
1977-01-01,AK: Anchorage,5342.24
1978-01-01,AK: Anchorage,5948.99
1979-01-01,AK: Anchorage,6158.68
1980-01-01,AK: Anchorage,5654.93
1981-01-01,AK: Anchorage,6192.83


In [4]:
df_TS_pivot = df_TS.pivot(columns='state_city', values='rev_total')
df_TS_pivot.head()

state_city,AK: Anchorage,AK: Fairbanks,AL: Birmingham,AL: Gadsden,AL: Mobile,AL: Montgomery,AR: Ft. Smith,AR: Little Rock,AR: Pine Bluff,AZ: Mesa,...,WA: Seattle,WA: Spokane,WA: Tacoma,WI: Madison,WI: Milwaukee,WV: Charleston,WV: Huntington,WV: Wheeling,WY: Casper,WY: Cheyenne
year,,,,,,,,,,,,,,,,,,,,,
1977-01-01,5342.24,10694.10,3585.56,2110.24,2422.69,2404.86,1974.81,2595.48,1843.03,4141.52,...,5087.61,3340.49,5764.70,3694.34,4962.91,3267.59,2719.36,2615.27,3565.21,3371.69
1978-01-01,5948.99,9879.05,3812.13,2421.50,2409.94,2414.52,2102.94,2603.06,1838.35,4486.91,...,5296.63,3019.63,6132.73,3628.95,5291.06,2997.67,2589.10,2710.00,3874.99,3655.38
1979-01-01,6158.68,9956.39,3683.44,2495.91,2518.27,2339.53,2014.04,2569.56,1815.43,4393.78,...,5200.40,3063.28,6082.74,3415.57,5194.10,3371.48,2435.91,2545.52,3827.20,3834.86
1980-01-01,5654.93,9228.33,3413.75,2319.35,2332.95,2208.98,1907.19,2339.04,1807.27,4353.79,...,4873.79,3016.78,5530.47,3268.03,4859.10,2880.31,2574.81,2391.60,4053.25,3653.97
1981-01-01,6192.83,9846.12,3354.20,2498.66,2400.45,2120.81,1994.07,2571.07,2172.49,4267.93,...,4848.54,2942.93,5493.17,3414.55,4657.09,3090.90,2868.05,2848.57,4698.07,3790.32


In [5]:
df_TS_pivot.index = pd.DatetimeIndex(df_TS_pivot.index.values,freq=df_TS_pivot.index.inferred_freq)
df_TS_pivot.index

DatetimeIndex(['1977-01-01', '1978-01-01', '1979-01-01', '1980-01-01',
               '1981-01-01', '1982-01-01', '1983-01-01', '1984-01-01',
               '1985-01-01', '1986-01-01', '1987-01-01', '1988-01-01',
               '1989-01-01', '1990-01-01', '1991-01-01', '1992-01-01',
               '1993-01-01', '1994-01-01', '1995-01-01', '1996-01-01',
               '1997-01-01', '1998-01-01', '1999-01-01', '2000-01-01',
               '2001-01-01', '2002-01-01', '2003-01-01', '2004-01-01',
               '2005-01-01', '2006-01-01', '2007-01-01', '2008-01-01',
               '2009-01-01', '2010-01-01', '2011-01-01', '2012-01-01',
               '2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01',
               '2017-01-01'],
              dtype='datetime64[ns]', freq='AS-JAN')

## Create useful functions!
In order to facilitate/automate the training process, we create functions that we can later call in iterative loops.
Four specific functions are created below: 1) `check_adfuller`, 2) `check_mean_std`, 3) `evaluate_arima_model` and 4) `evaluate_models`.

The first two models can be used if we need to test for stationarity. The third is designed to train a specific ARIMA model and return the mean squared error of the test and the fourth model is designed to iterate through a set of ARIMA models to return the best model.

In [6]:
# check_adfuller
def check_adfuller(ts):
    # Dickey-Fuller test
    result = adfuller(ts, autolag='AIC')
    print('Test statistic: ' , result[0])
    print('p-value: '  ,result[1])
    print('Critical Values:' ,result[4])
    
# check_mean_std
def check_mean_std(ts):
    #Rolling statistics
    rolmean = ts.rolling(3).mean()
    rolstd = ts.rolling(3).std()
    plt.figure(figsize=(22,10))   
    orig = plt.plot(ts, color='red',label='Original')
    mean = plt.plot(rolmean, color='black', label='Rolling Mean')
    std = plt.plot(rolstd, color='green', label = 'Rolling Std')
    plt.xlabel("Date")
    plt.ylabel("Mean Temperature")
    plt.title('Rolling Mean & Standard Deviation')
    plt.legend()
    plt.show()

### Details about the evaluation of each ARIMA model
The following function is meant to test each ARIMA model. To do so, the dataset is first made stationary using the log-difference method. To evaluate the performance of each ARIMA model, the dataset is split into a training section and a testing section (85%-15% split). Since this is a timeseries, a marching approach is adopted. Explicitly, a marching approach means that the model will use the first 85% of the dataset, to forecast the next data point, then add this forecast to the training set, and repeat the process until a forecast is generate for each data point in the testing set.

Before checking how the model performed, the forecasted data is first transformed back to a non-stationary form. The performance of the ARIMA model is evaluated using two metrics: 1) the mean squared error and 2) the absolute percent error, between the forecasted values and the testing values.

In [29]:
# A function called evaluate_arima_model to find the MSE of a single ARIMA model 
def evaluate_arima_model(data, arima_order):
    # Make data stationary
    data_stationary = (np.log(data)).diff().dropna()
    
    split=int(len(data_stationary) * 0.85) 
    # Make train and test variables, with 'train, test'
    train, test = data_stationary[0:split], data_stationary[split:len(data_stationary)]
    past=[x for x in train]
    # make predictions
    predictions = list()
    for i in range(len(test)):#timestep-wise comparison between test data and one-step prediction ARIMA model. 
        model = ARIMA(past, order=arima_order)
        model_fit = model.fit(disp=0)
        future = model_fit.forecast()[0]
        predictions.append(future)
        past.append(test[i])
    # calculate out of sample error
    error = mean_squared_error(test, predictions)
    
    # convert forecasted time series back to stationay form
    last = data[0:-len(test)][-1]
    predictions_unstationary=[None] * len(test)
    for i in range(len(test)):
        predictions_unstationary[i]= np.exp(predictions[i])*last
        last = predictions_unstationary[i]

    abs_pct_err = "{:.2%}".format ( (np.abs(data[-len(test):].values -  
                                        np.transpose(predictions_unstationary)) 
                                    / data[-len(test):].values ).mean())
    # Return the error
    return error, abs_pct_err

### Details choosing the best ARIMA model
The following function takes several different p, d, and q values to form different ARIMA models. The function iteratively evaluates each of these models, and then compares their performance. Finally the function returns the best function as well as the absolute % error metric calculated during the evaluation phase.

In [8]:
# Make a function called evaluate_models to evaluate different ARIMA models with several different p, d, and q values.
def evaluate_models(dataset, p_values, q_values):
    best_score, best_cfg = float("inf"), None
    # Iterate through p_values
    for p in p_values:
        # Iterate through d_values
         for d in d_values:
             # Iterate through q_values
            for q in q_values:
                # p, d, q iterator variables in that order
                order = (p,d,q)
                try:
                    # Make a variable called mse for the Mean squared error
                    mse, abs_pct_err = evaluate_arima_model(dataset, order)
                    if mse < best_score:
                        best_score, best_cfg = mse, order
                    #print('ARIMA%s MSE=%.3f' % (order,mse))
                except:
                    continue

    #print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score)) 
    return best_cfg, abs_pct_err

## Training, testing and picking the best ARIMA model
In order to generate different ARIMA models, a set of p, d and q values is first created. These are fed as input in the above ARIMA evaluation functions and for each city, the p-d-q combination that leads to the lowest mean squared error is picked. Therefore, each city will have an optimized ARIMA model based on its historical values.

In [9]:
# Now, we choose a couple of values to try for each parameter: p_values, d_values and q_values
# Fill in the blanks as appropriate
p_values = [x for x in range(0, 3)]
d_values = [x for x in range(0, 3)]
q_values = [x for x in range(0, 3)]

In [10]:
df_model_orders = pd.DataFrame(index=df_TS_pivot.columns, columns=['best_ARIMA_order', 'Training Error(%)'])

In [11]:
import warnings
warnings.filterwarnings("ignore")
print("Finding best ARIMA models for each city...")
for state_city in df_TS_pivot.columns:
    if list(df_TS_pivot.columns).index(state_city) % 15 == 0:
        print("{:.0%}".format(list(df_TS_pivot.columns).index(state_city) / len(df_TS_pivot.columns))+" done")
    dataset = df_TS_pivot[state_city]
    best_cfg, abs_pct_err = evaluate_models(dataset, p_values, q_values)
    df_model_orders.at[state_city, 'best_ARIMA_order'] =  best_cfg
    df_model_orders.at[state_city, 'Training Error(%)'] = abs_pct_err

Finding best ARIMA models for each city...
0% done
7% done
14% done
21% done
28% done
35% done
42% done
50% done
57% done
64% done
71% done
78% done
85% done
92% done
99% done


### Save best ARIMA model found for each city

In [34]:
df_model_orders.to_csv(r'/Users/Varishth/Desktop/Springboard_Projects/capstone_two/data/best_ARIMA_citywise.csv',index=False)
df_model_orders.head()

,best_ARIMA_order,Training Error(%),train_err_num
state_city,,,
AK: Anchorage,"(2, 1, 2)",30.58%,0.3058
AK: Fairbanks,"(0, 1, 2)",24.60%,0.2460
AL: Birmingham,"(2, 1, 1)",9.87%,0.0987
AL: Gadsden,"(1, 1, 1)",1.90%,0.0190
AL: Mobile,"(0, 1, 0)",8.75%,0.0875


## Predicting out-of-sample data and displaying the data
The following function serves two purposes:
* Firstly, it takes as input a city and uses the chosen ARIMA model for that city to predict and out-of-sample revenue for the city
* Secondly, the function also encodes visualization which will be later called in a widget to showcase the performance of the chosen ARIMA model for each city

In [13]:
def predict_city_revenue(chosen_state_city="IL: Chicago"):
    # fit model
    state_city = chosen_state_city
    actual_ts = (np.log(df_TS_pivot[state_city])).diff().dropna()
    arima_order = df_model_orders.loc[state_city].best_ARIMA_order
    split=int(len(actual_ts) * 0.85)

    # Make train and test variables, with 'train, test'
    train, test = actual_ts[0:split], actual_ts[split:len(actual_ts)]
    past=[x for x in train]
    # make predictions
    predictions = list()
    ci_lo = list()
    ci_hi = list()
    for i in range(len(test)):#timestep-wise comparison between test data and one-step prediction ARIMA model. 
        model = ARIMA(past, order=arima_order)
        model_fit = model.fit(disp=0)
        future = model_fit.forecast()[0]
        ci = model_fit.forecast()[2][0]
        predictions.append(future)
        ci_lo.append(ci[0])
        ci_hi.append(ci[1])
        past.append(test[i])

    # Out of sample forecast
    model = ARIMA(actual_ts, order=arima_order)
    model_fit = model.fit()
    forecast_ts = model_fit.forecast(alpha=0.05)

    # convert forecasted time series back to unstationary form
    last = df_TS_pivot[state_city][-1]
    forecast_ts_unstationary=[None] * 2
    forecast_ts_unstationary[0] = last
    forecast_ts_unstationary[1] = last * np.exp(forecast_ts[0])

    # convert to unstationary form
    last = df_TS_pivot[state_city][0:-len(test)][-1]
    predictions_unstationary=[None] * len(predictions)
    ci_lo_unstationary=[None] * len(ci_lo)
    ci_hi_unstationary=[None] * len(ci_hi)
    for i in range(len(predictions)):
        predictions_unstationary[i]= np.exp(predictions[i][0])*last
        ci_lo_unstationary[i] = np.exp(ci_lo[i])*last
        ci_hi_unstationary[i] = np.exp(ci_hi[i])*last
        last = predictions_unstationary[i]
        
    # pull in absolute error
    abs_pct_err = "{:.2%}".format ( (np.abs(df_TS_pivot[state_city][-len(test):].values -  
                                        np.transpose(predictions_unstationary)) 
                                    / df_TS_pivot[state_city][-len(test):].values ).mean())

    # visualization
    plt.figure(figsize=(10,5))
    plt.plot(df_TS_pivot[state_city], 'k',label = "Original")
    plt.plot(test.index, predictions_unstationary, 'k--',label='Step-wise Testing')
    plt.fill_between(test.index, ci_lo_unstationary, ci_hi_unstationary, 
                     color='k', alpha=.05,label='Step-wise Testing Error Bounds')
    plt.figtext(0.3, 0.59, "Absolute % error in testing = "+abs_pct_err, wrap=True, 
                horizontalalignment='center', fontsize=12)
    plt.plot(pd.to_datetime(['2017-01-01', '2018-01-01']), forecast_ts_unstationary, 'r--', label = "Forecast")
    plt.title("{} - Yearly Total Revenue".format(state_city), fontsize=14)
    plt.xlabel("Year")
    plt.ylabel("Total Revenue")
    plt.legend(loc='upper left', fontsize=12)
    plt.show()

### The worst performance...

In [30]:
df_model_orders['train_err_num'] = df_model_orders['Training Error(%)'].str.rstrip('%').astype(float) / 100
df_model_orders.sort_values(by=['train_err_num'], ascending=False).head()

,best_ARIMA_order,Training Error(%),train_err_num
state_city,,,
PA: Harrisburg,"(2, 0, 0)",923.82%,9.2382
MO: St. Louis,"(0, 1, 2)",184.24%,1.8424
CA: San Diego,"(0, 1, 2)",182.84%,1.8284
CO: Denver,"(2, 1, 2)",135.42%,1.3542
MA: Boston,"(0, 0, 0)",125.77%,1.2577


### The best performance...

In [33]:
df_model_orders.sort_values(by=['train_err_num']).head()

,best_ARIMA_order,Training Error(%),train_err_num
state_city,,,
AL: Gadsden,"(1, 1, 1)",1.90%,0.0190
AR: Little Rock,"(0, 2, 2)",2.22%,0.0222
SC: Charleston,"(0, 0, 1)",2.55%,0.0255
TX: El Paso,"(2, 1, 0)",2.64%,0.0264
MS: Jackson,"(2, 0, 2)",2.90%,0.0290


## Visualization widget
Below is a widget which allows you to toggle between cities, and easily visualize the training performance of the chosen ARIMA model, as well as, the out-of-sample prediction.

In [14]:
widgets.interact(predict_city_revenue, chosen_state_city=df_TS_pivot.columns);

interactive(children=(Dropdown(description='chosen_state_city', index=53, options=('AK: Anchorage', 'AK: Fairb…